In [2]:
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

from scipy.stats import binned_statistic
from common import make_env


plt.rcParams['figure.dpi'] = 256
plt.rcParams['text.usetex'] = True



In [3]:
def plot_network(env, ax):
    color = 'black'

    adjacency_matrix = np.vstack(env.graph.edges())
    # Set level colors
    level_col = {'retailer': 0,
                'distributor': 1,
                'manufacturer': 2,
                'raw_materials': 3}

    max_density = np.max([len(v) for v in env.levels.values()])
    node_coords = {}
    node_num = 1

    for i, (level, nodes) in enumerate(env.levels.items()):
        n = len(nodes)
        node_y = max_density / 2 if n == 1 else np.linspace(0, max_density, n)
        node_y = np.atleast_1d(node_y)
        ax.scatter(np.repeat(i, n), node_y, label=level, s=400, ec=color, fc='white', marker='o')

        for y in node_y:
            ax.annotate(r'$N_{}$'.format(node_num), xy=(i, y), ha='center', va='center', fontsize=10)
            node_coords[node_num] = (i, y)
            node_num += 1

    # Draw edges
    for node_num, v in node_coords.items():
        x, y = v
        sinks = adjacency_matrix[np.where(adjacency_matrix[:, 0]==node_num)][:, 1]
        for s in sinks:
            try:
                sink_coord = node_coords[s]
            except KeyError:
                continue
            
            x_ = np.hstack([x, sink_coord[0]])
            y_ = np.hstack([y, sink_coord[1]])
            ax.plot(x_, y_, color=color, linestyle='--', alpha=0.6, zorder=-10)

    ax.set_ylabel('Node')
    ax.set_yticks([0], [''])
    ax.set_xlabel('Level')
    ax.set_xticks(np.arange(len(env.levels)), [k for k in env.levels.keys()])

    ax.set_ymargin(0.2)





def plot_episode(env_name, algo_name, name):
    save_path = f'./data/{env_name}/{algo_name}/{name}/'
    env = make_env(env_name)

    fig, ax = plt.subplots(nrows=4, ncols=2)
    fig.set_size_inches(8, 8)

    plot_network(env, ax[0,0])

    df_names = ['D', 'X', 'R', 'P', 'Y']
    df_headers = [[0,1], [0], [0,1], [0], [0,1]]
    data = {}

    for i, df_name in enumerate(df_names):
        df = pd.read_csv(save_path + f'eval/3/{df_name}.csv', index_col=[0], header=df_headers[i])
        data[df_name] = df

    cmap = plt.get_cmap('tab20')

    for i, retailer_market in enumerate(data['D'].columns):
        retailer, market = retailer_market
        ax[0,1].plot(data['D'][retailer_market], linewidth=1.0, label=retailer_market)

    for i, node_num in enumerate(data['X'].columns):
        ax[1,0].plot(data['X'][node_num], linewidth=1.0, label=node_num, color=cmap(i))

    for i, supplier_requester in enumerate(data['R'].columns):
        supplier, requester = supplier_requester
        ax[1,1].plot(data['R'][supplier_requester], linewidth=1.0, label=supplier_requester, color=cmap(i))

    for i, node_num in enumerate(data['P'].columns):
        ax[2,0].plot(data['P'][node_num], linewidth=1.0, label=node_num, color=cmap(i))
        ax[2,1].plot(np.cumsum(data['P'][node_num]), linewidth=1.0, label=node_num, color=cmap(i))

    Rtot = data['R'].groupby('Requester', axis=1).sum()

    for i, node_num in enumerate(Rtot.columns):
        ax[3,0].plot(Rtot[node_num], linewidth=1.0, label=node_num, color=cmap(i))

    for i, source_reciever in enumerate(data['Y'].columns):
        source, reciever = source_reciever
        ax[3,1].plot(data['Y'][source_reciever], linewidth=1.0, label=source_reciever, color=cmap(i))

    ax[0,1].legend(fontsize=6)
    ax[1,0].legend(fontsize=6)
    ax[1,1].legend(fontsize=6)
    
    ax[0,1].set_xlabel('Timestep')
    ax[0,1].set_ylabel('Market Demand')

    ax[1,0].set_xlabel('Timestep')
    ax[1,0].set_ylabel('Inventory Node Stock')

    ax[1,1].set_xlabel('Timestep')
    ax[1,1].set_ylabel('Reorder Amount')

    ax[2,0].set_xlabel('Timestep')
    ax[2,0].set_ylabel('Node Profit')

    ax[2,1].set_xlabel('Timestep')
    ax[2,1].set_ylabel('Cumulaive Node Profit')


    fig.tight_layout(h_pad=0.0, w_pad=0.0)
    fig.show()


plot_episode('NetworkManagement-v1-100', 'TRPO', 'default')


/var/folders/v5/b2527n0x4zgb6khjjq0g22k40000gn/T/ipykernel_15677/1881659822.py:115: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


In [ ]:
def plot_training_curves(env_name, model_names):
    fig, ax = plt.subplots(nrows=2, ncols=1)
    fig.set_size_inches(8, 6)

    linewidth = 0.8
    alpha = 0.4

    cmap = plt.get_cmap('tab20')

    for i, model_name in enumerate(model_names):
        algorithm_name, name = model_name
        data = pd.read_csv(f'./data/{env_name}/{algorithm_name}/{name}/monitor.csv', skiprows=1)

        num_bins = 100
        ep_binned_mean_r = binned_statistic(np.log10(np.arange(len(data['r'])) + 1), data['r'], 'mean', num_bins)
        ep_binned_std_r = binned_statistic(np.log10(np.arange(len(data['r'])) + 1), data['r'], 'std', num_bins)
        
        ax[0].plot(ep_binned_mean_r.bin_edges[1:], ep_binned_mean_r.statistic, 
            linewidth=linewidth, label=algorithm_name + ' ' + name, color=cmap(i))
        ax[0].plot(ep_binned_mean_r.bin_edges[1:], ep_binned_mean_r.statistic + ep_binned_std_r.statistic, 
            linewidth=linewidth, color=cmap(i), linestyle='--', alpha=alpha)
        ax[0].plot(ep_binned_mean_r.bin_edges[1:], ep_binned_mean_r.statistic - ep_binned_std_r.statistic, 
            linewidth=linewidth, color=cmap(i), linestyle='--', alpha=alpha)

        num_bins = 100
        t_binned_mean_r = binned_statistic(np.log10(data['t']), data['r'], 'mean', num_bins)
        t_binned_std_r = binned_statistic(np.log10(data['t']), data['r'], 'std', num_bins)

        ax[1].plot(t_binned_mean_r.bin_edges[1:], t_binned_mean_r.statistic, 
            linewidth=linewidth, label=algorithm_name + ' ' + name, color=cmap(i))
        ax[1].plot(t_binned_mean_r.bin_edges[1:], t_binned_mean_r.statistic + ep_binned_std_r.statistic, 
            linewidth=linewidth, color=cmap(i), linestyle='--', alpha=alpha)
        ax[1].plot(t_binned_mean_r.bin_edges[1:], t_binned_mean_r.statistic - ep_binned_std_r.statistic, 
            linewidth=linewidth, color=cmap(i), linestyle='--', alpha=alpha)

        # ax[1].plot(data['t'], data['r'], linewidth=linewidth, alpha=0.4, color=colors[i]))

    ax[0].legend(fontsize=6)

    ax[0].set_xlabel('Log Episode')
    ax[0].set_ylabel('Reward')

    ax[1].set_xlabel('Log Walltime')
    ax[1].set_ylabel('Reward')

    fig.tight_layout()
    fig.show()

models = [('PPO', 'default'), ('TRPO', 'default'), ('RecurrentPPO', 'default')]

plot_training_curves('NetworkManagement-v1-100', models)

NameError: name 'plt' is not defined

In [ ]:
def plot_eval_episodes(env_name, model_names):
    fig, ax = plt.subplots(nrows=1, ncols=2)
    fig.set_size_inches(8, 3)
    
    linewidth = 0.6
    alpha = 0.4

    def plot_rewards(rewards, label, color):
        reward_means = np.mean(rewards, axis=0)
        reward_stds = np.std(rewards, axis=0)

        ax[0].plot(reward_means, linewidth=linewidth, label=label, color=color)
        ax[0].plot(reward_means + reward_stds, linewidth=linewidth, color=color, linestyle='--', alpha=alpha)
        ax[0].plot(reward_means - reward_stds, linewidth=linewidth, color=color, linestyle='--', alpha=alpha)

        cumulative_rewards = np.cumsum(rewards, axis=1)
        cumulative_reward_means = np.mean(cumulative_rewards, axis=0)
        cumulative_reward_stds = np.std(cumulative_rewards, axis=0)

        ax[1].plot(cumulative_reward_means, linewidth=linewidth, label=label, color=color)
        ax[1].plot(cumulative_reward_means + cumulative_reward_stds, linewidth=linewidth, color=color, linestyle='--', alpha=alpha)
        ax[1].plot(cumulative_reward_means - cumulative_reward_stds, linewidth=linewidth, color=color, linestyle='--', alpha=alpha)

    cmap = plt.get_cmap('tab20')

    for i, model_name in enumerate(model_names):
        algo_name, name = model_name
        rewards = np.load(f'./data/{env_name}/{algo_name}/{name}/best_model_eval_rewards.npy')

        plot_rewards(rewards, algo_name + ' ' + name, cmap(i))
        
    ax[0].legend(fontsize=6)
    
    ax[0].set_xlabel('Episode')
    ax[0].set_ylabel('Reward')

    ax[1].set_xlabel('Episode')
    ax[1].set_ylabel('Cumulative Reward')

    fig.tight_layout()
    fig.show()